# Singular Value Decomposition - Jacobi and Lanczos Methods

---

Since computing the SVD of $A$ can be seen as computing the EVD of the symmetric matrices 
$A^*A$, $AA^*$, or $\begin{bmatrix}0 & A \\ A^* & 0 \end{bmatrix}$, simple modifications of the corresponding EVD algorithms yield version for comuting the SVD.

For more details on one-sided Jacobi method, see 
[Z. Drmač, Computing Eigenvalues and Singular Values to High Relative Accuracy][Hog14a]
and the references therein.

[Hog14a]: #1 "L. Hogben, ed., 'Handbook of Linear Algebra', pp. 59.1-59.21, CRC Press, Boca Raton, 2014."


## Prerequisites

The reader should be familiar with concepts of singular values and vectors, related perturbation theory, and algorithms, and Jacobi and Lanczos methods for the symmetric eigenvalue decomposition.

 
## Competences 

The reader should be able to recognise matrices which warrant high relative accuracy and to apply 
Jacobi method to them. The reader should be able to recognise matrices to which Lanczos method can be efficiently applied and do so.

---

## One-sided Jacobi method

Let $A\in\mathbb{R}^{m\times n}$ with $\mathop{\mathrm{rank}}(A)=n$ (therefore, $m\geq n$) and
$A=U\Sigma V^T$ its thin SVD.

### Definition

Let $A=BD$, where $D=\mathop{\mathrm{diag}} (\| A_{:,1}\|_2, \ldots, \|A_{:,n}\|_2)$ is a __diagonal scaling__ , and $B$ is the __scaled matrix__ of $A$ from the right. 
Then $[B^T B]_{i,i}=1$.

### Facts

1. Let $\tilde U$, $\tilde V$ and $\tilde \Sigma$ be the approximations of $U$, $V$ and $\Sigma$, respectively, computed by a backward stable algorithm as 
$A+\Delta A=\tilde U\tilde \Sigma \tilde V^T$. Since the orthogonality of $\tilde U$ and $\tilde V$ cannot be guaranteed, this product in general does not represent and SVD. There exist nearby orthogonal matrices $\hat U$ and $\hat V$ such that 
$(I+E_1)(A+\Delta A)(I+E_2)=\hat U \tilde \Sigma \hat V^T$, where departures from orthogonalithy, 
$E_1$ and $E_2$, are small in norm.

2. Standard algorithms compute the singular values with backward error 
$\| \Delta A\|\leq \phi\varepsilon \|A\|_2$, where $\varepsilon$ is machine precision and 
$\phi$ is a slowly growing function og $n$. The best error bound for the singular values is 
$|\sigma_j-\tilde \sigma_j|\leq \| \Delta A\|_2$, and the best relative error bound is
$$
\max_j \frac{|\sigma_j-\tilde\sigma_j|}{\sigma_j}\leq \frac{\| \Delta A \|_2}{\sigma_j} \leq
\phi \varepsilon \kappa_2(A).
$$

3. Let $\|[\Delta A]_{:,j}\|_2\leq \varepsilon \| A_{:,j}\|_2$ for all $j$. Then
$A+\Delta A=(B+\Delta B)D$ and $\|\Delta B\|_F\leq \sqrt{n}\varepsilon$, and
$$
\max_j \frac{|\sigma_j-\tilde\sigma_j|}{\sigma_j}\leq 
\| (\Delta B) B^{\dagger} \|_2\leq
\sqrt{n}\varepsilon \| B^{\dagger}\|_2.
$$
This is Fact 3 from the [Relative perturbation theory](L5b Singular Value Decomposition - Perturbation Theory .ipynb).

4. It holds
$$
\| B^{\dagger} \| \leq \kappa_2(B) \leq \sqrt{n} \min_{S=\mathop{\mathrm{diag}}} 
\kappa_2(AS)\leq \sqrt{n}\kappa_2(A).
$$
Therefore, numerical algorithm with column-wise small backward error computes singular values more accurately than an algorithm with small norm-wise backward error.

5. In each step, one-sided Jacobi method computes the Jacobi roataion matrix from the pivot submatrix of the current matrix $A^TA$. Afterwards, $A$ is multiplied with the computed rotation matrix from the right (only two columns are affected). 
Convergence of the Jacobi method for the symmetric matrix $A^TA$ to a diagonal matrix, implies that the matrix $A$ converges to the matrix $AV$ with orthogonal columns and $V^TV=I$. 
Then $AV=U\Sigma$, $\Sigma=\mathop{\mathrm{diag}}(\| A_{:,1}\|_2, \ldots, \| A_{:,n}\|_2)$, $U=AV\Sigma^{-1}$, and  $A=U\Sigma V^T$ is the SVD of $A$.

6. One-sided Jacobi method computes the SVD with error bound from Facts 2 and 3, provided that the condition of the intermittent scaled matrices does not grow much. There is overwhelming numerical evidence for this. Alternatively, 
if $A$ is square, the one-sided Jacobi method can be applied to the transposed matrix $A^T=DB^T$ and the same error bounds apply, but the condition of the scaled matrix  (_this time from the left_) does not change. This approach is slower.

7. One-sided Jacobi method can be preconditioned by applying one QR factorization with full pivoting and one QR factorization withour pivoting to $A$, to obtain faster convergence, without sacrifying accuracy. This method is implemented in the LAPACK routine 
[DGESVJ](http://www.netlib.org/lapack/explore-html-3.3.1/d1/d5e/dgesvj_8f_source.html).
_Writing the wrapper for `DGESVJ` is a tutorial assignment._

### Example - Standard matrix

In [1]:
function myJacobiR{T}(A::Array{T})
    m,n=size(A)
    V=eye(T,n,n)
    # Tolerance for rotation
    tol=sqrt(n)*eps(T)
    # Counters
    p=n*(n-1)/2
    sweep=0
    pcurrent=0
    # First criterion is for standard accuracy, second one is for relative accuracy
    # while sweep<30 && vecnorm(A-diagm(diag(A)))>tol
    while sweep<30 && pcurrent<p
        sweep+=1
        # Row-cyclic strategy
        for i = 1 : n-1 
            for j = i+1 : n
                # Compute the 2 x 2 sumbatrix of A'*A
                F=A[:,[i,j]]'*A[:,[i,j]]
                # Check the tolerance - the first criterion is standard,
                # the second one is for relative accuracy               
                # if A[i,j]!=zero(T)
                # 
                if abs(F[1,2])>tol*sqrt(F[1,1]*F[2,2])
                    # Compute c and s
                    τ=(F[1,1]-F[2,2])/(2*F[1,2])
                    t=sign(τ)/(abs(τ)+sqrt(1+τ^2))
                    c=1/sqrt(1+t^2)
                    s=c*t
                    G=LinAlg.Givens(i,j,c,s)
                    A*=G'
                    V*=G'
                    pcurrent=0
                else
                    pcurrent+=1
                end
            end
        end
    end
    σ=[vecnorm(A[:,k]) for k=1:n]
    for k=1:n
        A[:,k]=A[:,k]/σ[k]
    end
    A, σ, V
end

myJacobiR (generic function with 1 method)

In [2]:
m=8
n=5
A=map(Float64,rand(-9:9,m,n))

8×5 Array{Float64,2}:
  7.0  -6.0  -2.0   1.0   4.0
 -6.0  -4.0  -3.0   2.0   2.0
 -8.0   7.0   2.0   3.0   3.0
 -2.0   7.0   0.0   9.0   3.0
  8.0   8.0   1.0   6.0   3.0
 -4.0   9.0   8.0  -1.0  -6.0
  6.0   1.0  -2.0   6.0  -3.0
  3.0  -3.0   4.0  -1.0  -5.0

In [3]:
U,σ,V=myJacobiR(A)

(
[0.338691 -0.391497 … 0.0881288 0.200217; -0.196746 -0.070242 … 0.487893 -0.410713; … ; 0.448586 -0.00436256 … -0.14364 -0.663759; -0.0386993 -0.122365 … -0.460213 -0.352135],

[17.1983,20.3903,3.86253,14.0572,6.85054],
[0.761015 -0.371915 … -0.488607 0.206211; 0.211478 0.822336 … -0.138851 0.302679; … ; 0.547568 0.303463 … 0.331796 -0.63343; 0.223881 -0.0354554 … 0.633379 0.666299])

In [4]:
# Residual 
A*V-U*diagm(σ)

8×5 Array{Float64,2}:
  3.55271e-15  -2.66454e-15  -1.77636e-15   4.44089e-16  -2.22045e-16
 -4.44089e-16  -2.22045e-16  -1.77636e-15  -1.77636e-15   0.0        
 -3.55271e-15   0.0           4.44089e-16   1.77636e-15   7.77156e-16
 -3.55271e-15   0.0           3.33067e-16  -1.77636e-15   2.44249e-15
 -3.55271e-15   1.77636e-15  -2.77556e-17  -4.44089e-16   8.88178e-16
 -2.66454e-15  -5.32907e-15   2.9976e-15   -8.88178e-16   4.44089e-16
 -1.77636e-15   2.35922e-16   0.0          -1.77636e-15   1.77636e-15
  8.88178e-16  -1.33227e-15   2.22045e-15  -1.77636e-15  -4.44089e-16

### Example - Strongly scaled matrix

In [5]:
m=20
n=15
B=rand(m,n)
D=exp(50*(rand(n)-0.5))
As=B*diagm(D)

20×15 Array{Float64,2}:
 1.78045e9  1.64648e-8  0.0560665   …  9.4779e-9    1.05534e-6  1.72284e6
 2.05122e9  1.7941e-8   0.0586242      9.89661e-9   2.48788e-6  6.84764e5
 2.01641e9  1.89669e-8  0.0477597      2.69136e-10  1.48552e-6  3.72601e6
 2.41445e8  9.95544e-9  0.00513408     1.23753e-8   1.54953e-6  3.79646e6
 2.01702e9  4.79065e-9  0.0610346      2.00124e-9   1.20745e-6  1.77584e6
 1.41192e9  4.06006e-9  0.0291878   …  3.08367e-9   1.18237e-6  3.33596e6
 2.41633e9  1.2565e-8   0.0533464      1.44826e-8   2.57155e-6  3.40213e6
 1.31774e9  1.62315e-8  0.0650711      1.83629e-8   7.74347e-7  2.55604e6
 6.8028e8   2.44321e-9  0.0340971      9.27077e-9   5.04685e-7  8.32925e5
 1.75412e9  1.89904e-8  0.0699813      1.42108e-8   1.71649e-6  6.60291e5
 1.79641e8  1.70623e-8  0.0628472   …  3.05038e-10  1.57233e-6  1.7083e6 
 7.5232e7   7.48443e-9  0.0241104      3.34577e-9   2.6477e-6   3.69597e5
 1.13624e9  8.17795e-9  0.0639473      4.05529e-9   1.62323e-6  4.06373e5
 1.6125e8   1.

In [6]:
cond(B), cond(As)

(26.152071415315845,3.692988050816275e20)

In [7]:
Us,σs,Vs=myJacobiR(As);

In [8]:
[sort(σs,rev=true) svdvals(As)]

15×2 Array{Float64,2}:
  6.71791e10    6.71791e10 
  4.15613e9     4.15613e9  
  5.06247e8     5.06247e8  
  3.65619e8     3.65619e8  
  5.54928e6     5.54928e6  
  3.95591e6     3.95591e6  
 13.5562       13.5562     
  0.057207      0.057207   
  0.00279086    0.00279086 
  0.00083445    0.00083445 
  2.72712e-6    3.32688e-6 
  2.71519e-8    1.41734e-6 
  1.67331e-8    2.27059e-8 
  4.51809e-10   6.09989e-10
  2.84314e-11   1.8191e-10 

In [9]:
(sort(σs,rev=true)-svdvals(As))./sort(σs,rev=true)

15-element Array{Float64,1}:
   1.13568e-16
  -1.14731e-16
   1.17738e-16
  -1.63024e-16
   0.0        
   2.35426e-16
   2.62073e-15
   1.52918e-7 
  -1.80033e-7 
  -1.95397e-7 
  -0.219921   
 -51.2004     
  -0.35695    
  -0.350103   
  -5.3982     

In [10]:
norm(As*Vs-Us*diagm(σs))

1.8447252518771906e-5

In [11]:
Us'*As*Vs

15×15 Array{Float64,2}:
  4.15613e9   -1.12865e-23  -4.55886e-18  …  -5.83059e-22   3.18985e-10
 -3.90552e-8   1.67331e-8   -7.78743e-19     -5.35584e-22   7.59661e-11
  2.13146e-7   9.31624e-25   0.057207         5.74557e-22   7.95656e-10
  2.04474e-7  -2.22528e-24   1.46124e-17      2.34417e-21   1.13873e-9 
  3.0569e-7    1.42569e-25   4.95385e-18     -8.24815e-22  -1.62986e-10
 -4.00592e-7  -3.23454e-25   5.17284e-18  …   6.91535e-22   2.6027e-10 
  1.63981e-7  -8.66029e-24  -2.47069e-17      3.22583e-22   2.16424e-10
  2.13561e-7  -1.57432e-24   1.12921e-17     -7.11304e-22   5.03604e-11
 -1.54876e-6   3.80339e-24  -5.15842e-18     -4.85435e-21  -8.38717e-10
 -4.33798e-7   1.45398e-24   1.97483e-18     -2.76156e-22   5.9336e-10 
 -4.14776e-7   5.65628e-24   1.39968e-17  …  -2.45386e-22   1.04235e-10
  1.71484e-6   9.10678e-24  -4.79192e-19     -7.16738e-22   5.23212e-10
  1.13077e-7   6.11031e-24   4.78089e-18      9.541e-22    -9.03441e-10
  2.00316e-7  -3.25857e-25   1.91886e-18

In the alternative approach, we first apply QR factorization with column pivoting to obtain the square matrix.

In [12]:
Q,R,p=qr(As,Val{true},thin=true)

(
[-0.046866 -0.375308 … -0.222352 0.592707; -0.193881 -0.272682 … 0.456548 -0.185945; … ; -0.192395 0.0333 … 0.31692 0.153577; -0.0867368 -0.359395 … 0.179608 -0.133907],

[-6.70027e10 -4.74017e9 … -1.27747e-9 -7.14937e-11; 0.0 -4.15205e9 … -3.33126e-10 -2.95983e-11; … ; 0.0 0.0 … 4.51538e-10 1.84088e-11; 0.0 0.0 … 0.0 -2.84552e-11],

[9,1,12,5,15,10,4,3,7,11,14,13,2,8,6])

In [13]:
diag(R)

15-element Array{Float64,1}:
 -6.70027e10 
 -4.15205e9  
  4.87841e8  
 -3.8078e8   
  5.44599e6  
 -4.03099e6  
 13.5562     
 -0.0572018  
 -0.00278714 
 -0.00083564 
 -2.7271e-6  
  2.27597e-8 
  1.99577e-8 
  4.51538e-10
 -2.84552e-11

In [14]:
UR,σR,VR=myJacobiR(R')

(
[-0.997365 0.0719065 … -4.77242e-21 -8.57992e-23; -0.0708351 -0.993872 … -1.62399e-20 3.5787e-21; … ; -1.90384e-20 -7.97376e-20 … 0.99893 0.0408972; -1.06622e-21 -7.10765e-21 … 0.0408899 -0.999163],

[6.71791e10,4.15613e9,5.06247e8,3.65619e8,5.54928e6,3.95591e6,13.5562,0.057207,0.00279086,0.00083445,2.72712e-6,2.71519e-8,1.67331e-8,4.51809e-10,2.84314e-11],
[0.99999 -0.00446031 … 3.21811e-41 3.64074e-44; 0.00445938 0.99994 … 1.73042e-39 -2.45469e-41; … ; 0.0 2.4001e-42 … 0.999997 0.00257526; 0.0 -1.59535e-44 … -0.00257526 0.999997])

In [15]:
(sort(σs)-sort(σR))./sort(σs)

15-element Array{Float64,1}:
 -2.27296e-16
  4.57705e-16
  1.97736e-15
  4.87439e-16
  9.31787e-16
  3.89791e-16
  6.21573e-16
  4.85178e-16
  9.17256e-16
  0.0        
 -3.35656e-16
  0.0        
  1.17738e-16
  1.14731e-16
 -1.13568e-16

In [16]:
P=eye(15)
P=P[:,p];

Now $QRP^T=A$ and $R^T=U_R\Sigma_R V^T_R$, so $A=(Q V_R) \Sigma_R (U_R^T P^T)$ is an SVD of $A$.

In [17]:
# Check the residual
U1=Q*VR
V1=UR[invperm(p),:]
norm(As*V1-U1*diagm(σR))

5.876005057927066e-5

## Lanczos method

The function `svds()` is based on the Lanczos method for symmetric matrices. Input can be matrix, but also an operator which defines the product of the given matrix with a vector.

In [18]:
?svds

search: svds svdvals svdvals! svd svdfact svdfact! isvalid



```
svds(A; nsv=6, ritzvec=true, tol=0.0, maxiter=1000, ncv=2*nsv, u0=zeros((0,)), v0=zeros((0,))) -> (SVD([left_sv,] s, [right_sv,]), nconv, niter, nmult, resid)
```

Computes the largest singular values `s` of `A` using implicitly restarted Lanczos iterations derived from [`eigs`](:func:`eigs`).

**Inputs**

  * `A`: Linear operator whose singular values are desired. `A` may be represented as a subtype of `AbstractArray`, e.g., a sparse matrix, or any other type supporting the four methods `size(A)`, `eltype(A)`, `A * vector`, and `A' * vector`.
  * `nsv`: Number of singular values. Default: 6.
  * `ritzvec`: If `true`, return the left and right singular vectors `left_sv` and `right_sv`.  If `false`, omit the singular vectors. Default: `true`.
  * `tol`: tolerance, see [`eigs`](:func:`eigs`).
  * `maxiter`: Maximum number of iterations, see [`eigs`](:func:`eigs`). Default: 1000.
  * `ncv`: Maximum size of the Krylov subspace, see [`eigs`](:func:`eigs`) (there called `nev`). Default: `2*nsv`.
  * `u0`: Initial guess for the first left Krylov vector. It may have length `m` (the first dimension of `A`), or 0.
  * `v0`: Initial guess for the first right Krylov vector. It may have length `n` (the second dimension of `A`), or 0.

**Outputs**

  * `svd`: An `SVD` object containing the left singular vectors, the requested values, and the right singular vectors. If `ritzvec = false`, the left and right singular vectors will be empty.
  * `nconv`: Number of converged singular values.
  * `niter`: Number of iterations.
  * `nmult`: Number of matrix–vector products used.
  * `resid`: Final residual vector.

**Example**

```julia
X = sprand(10, 5, 0.2)
svds(X, nsv = 2)
```

**Implementation note**

`svds(A)` is formally equivalent to calling `eigs` to perform implicitly restarted Lanczos tridiagonalization on the Hermitian matrix $\begin{pmatrix} 0 & A^\prime \\ A & 0 \end{pmatrix}$, whose eigenvalues are plus and minus the singular values of $A$.


In [19]:
m=20
n=15
A=rand(m,n);

In [20]:
U,σ,V=svd(A);

In [21]:
# All singular values
σL,rest=svds(A,nsv=15);

In [22]:
typeof(σL)

Base.LinAlg.SVD{Float64,Float64,Array{Float64,2}}

In [23]:
σL[:U]

20×15 Array{Float64,2}:
 0.225961   0.160487     0.0389855  …   0.054393    -0.405673    0.0138695
 0.207483  -0.328476     0.0561244      0.280715     0.425752    0.166213 
 0.286828   0.155198    -0.134651       0.0155462    0.0201997   0.305267 
 0.150792  -0.132273     0.0573987     -0.417034     0.1062      0.131586 
 0.204336   0.148386     0.255754      -0.191559    -0.159986   -0.356458 
 0.253166   0.00893541  -0.325043   …   0.139814    -0.0962878   0.0761997
 0.210497  -0.329848     0.274489       0.306031    -0.339515    0.154818 
 0.241144  -0.132862     0.0384068     -0.245493    -0.0500416   0.261145 
 0.229242   0.0132448    0.404024      -0.00861873   0.022737   -0.0277082
 0.188514   0.0269083    0.251751      -0.0316821    0.322149   -0.178608 
 0.182593   0.0754906   -0.2137     …   0.268876     0.282654    0.162064 
 0.261422  -0.160789     0.0347714     -0.336229    -0.0736459  -0.272997 
 0.233539  -0.129211    -0.395192      -0.154525     0.0343121  -0.231532 
 

In [24]:
σL[:S]

15-element Array{Float64,1}:
 9.06285 
 2.26399 
 1.77373 
 1.66818 
 1.50714 
 1.35708 
 1.33233 
 1.12165 
 1.00898 
 0.834469
 0.810611
 0.714304
 0.639772
 0.507688
 0.351793

In [25]:
(σ-σL[:S])./σ

15-element Array{Float64,1}:
 -1.96004e-16
 -3.92307e-16
 -1.00148e-15
  1.33106e-16
  8.83969e-16
  1.63619e-16
  0.0        
 -5.93888e-16
 -6.60204e-16
 -5.32182e-16
 -1.36961e-16
 -3.10854e-16
 -8.67671e-16
 -6.56047e-16
 -1.73574e-15

In [26]:
# Some largest singular values
σp,rest=svds(A,nsv=5);
(σ[1:5]-σp[:S])./σ[1:5]

5-element Array{Float64,1}:
 -1.96004e-16
 -5.8846e-16 
  1.00148e-15
 -9.3174e-16 
  4.41984e-16

### Example - Large matrix

In [27]:
m=2000
n=1500
Ab=rand(m,n);

In [28]:
@time Ub,σb,Vb=svd(Ab);

  4.738963 seconds (154 allocations: 131.799 MB, 1.97% gc time)


In [29]:
# This is rather slow
@time σl,rest=svds(Ab,nsv=10);

  2.756231 seconds (5.64 k allocations: 2.637 MB)


In [30]:
(σb[1:10]-σl[:S])./σb[1:10]

10-element Array{Float64,1}:
 -2.75645e-15
  1.48253e-16
  3.87227e-15
 -1.49526e-15
  7.48299e-16
  2.2515e-15 
  2.40974e-15
 -3.47678e-15
 -1.21022e-15
 -1.96869e-15

### Example - Very large sparse matrix

In [31]:
?sprand

search: sprand sprandn sprandbool StepRange



```
sprand([rng],[type],m,[n],p::AbstractFloat,[rfn])
```

Create a random length `m` sparse vector or `m` by `n` sparse matrix, in which the probability of any element being nonzero is independently given by `p` (and hence the mean density of nonzeros is also exactly `p`). Nonzero values are sampled from the distribution specified by `rfn` and have the type `type`. The uniform distribution is used in case `rfn` is not specified. The optional `rng` argument specifies a random number generator, see [Random Numbers](:ref:`Random Numbers <random-numbers>`).


In [32]:
m=10000
n=3000
A=sprand(m,n,0.05)

10000×3000 sparse matrix with 1498247 Float64 nonzero entries:
	[7    ,     1]  =  0.701342
	[14   ,     1]  =  0.856633
	[36   ,     1]  =  0.0916545
	[40   ,     1]  =  0.894791
	[59   ,     1]  =  0.162841
	[88   ,     1]  =  0.510488
	[180  ,     1]  =  0.611854
	[193  ,     1]  =  0.528632
	[206  ,     1]  =  0.489766
	[217  ,     1]  =  0.327671
	⋮
	[9834 ,  3000]  =  0.858184
	[9840 ,  3000]  =  0.769769
	[9864 ,  3000]  =  0.676939
	[9893 ,  3000]  =  0.247848
	[9922 ,  3000]  =  0.0749576
	[9949 ,  3000]  =  0.364391
	[9955 ,  3000]  =  0.420789
	[9964 ,  3000]  =  0.523681
	[9975 ,  3000]  =  0.565004
	[9988 ,  3000]  =  0.854927
	[9996 ,  3000]  =  0.709252

In [33]:
# No vectors, this takess about 30 sec.
@time σ1,rest=svds(A,nsv=100,ritzvec=false)

 35.474279 seconds (1.66 M allocations: 105.665 MB, 0.06% gc time)


(Base.LinAlg.SVD{Float64,Float64,Array{Float64,2}}(,[137.578,19.642,19.6214,19.5368,19.5056,19.4839,19.4737,19.4669,19.4568,19.423  …  18.5429,18.5404,18.5398,18.5119,18.5055,18.5012,18.4947,18.4906,18.4741,18.4683],),200,10,1563,[0.0371594,0.00916456,-0.0336012,-0.0546321,0.0177772,-0.059385,-0.0316879,0.0877661,0.0858211,-0.0258369  …  0.0709013,-0.0485231,0.108174,0.0889936,-0.0648916,0.0122142,0.021629,0.0791618,0.107022,-0.116599])

In [34]:
@time σ2=svdvals(full(A));

 23.243175 seconds (6.96 k allocations: 459.805 MB, 0.49% gc time)


In [35]:
(σ1[:S]-σ2[1:100])./σ2[1:100]

100-element Array{Float64,1}:
  6.19757e-16
 -6.33057e-15
  1.1588e-14 
 -3.27325e-15
 -1.42068e-14
 -1.27639e-14
 -2.91898e-15
 -1.04025e-14
 -1.82595e-16
 -4.207e-15  
  2.56186e-15
  7.50785e-15
 -4.40085e-15
  ⋮          
 -9.57102e-16
 -2.10626e-15
 -5.74781e-16
 -3.25755e-15
  1.14976e-15
 -5.75745e-16
  1.15189e-15
  2.68837e-15
  5.76282e-16
  3.84273e-16
 -1.34615e-15
  1.92369e-16